# Module 2: The Era of the Quarterback

## "The NFL Has Changed. Let's Prove It."

The narrative is: **The NFL is a passing league now.**

But as data scientists, we don't trust narratives — **we trust data.**

Let's prove it.


In [ ]:
# Standard imports
import polars as pl
from pathlib import Path
import sys

# Project setup
PROJECT_ROOT = Path.cwd()
sys.path.insert(0, str(PROJECT_ROOT / "src"))

from ingestion import load_pbp_cached
from viz import plot_epa_evolution, apply_dark_theme

import plotly.express as px
import plotly.graph_objects as go

# Load data
pbp = load_pbp_cached(seasons=range(2000, 2025))
print("✅ Data loaded")


## 🖐️ HANDS ON: Build a Polars Query Together

This is the **bread-and-butter Polars pattern**.

Type along with me — we'll build this query step by step:


In [ ]:
# HANDS ON
season_passing = (
    pbp
    .filter(pl.col("play_type") == "pass")
    .group_by("season")
    .agg([
        pl.col("epa").mean().alias("epa_per_play")
    ])
    .sort("season")
    .collect()
)

print("League-Wide Passing Efficiency by Season:")
season_passing

## Visualizing the Trend

Let's see this data come alive:


In [ ]:
# Visualize the passing efficiency trend
fig = plot_epa_evolution(
    season_passing,
    x_col="season",
    y_col="epa_per_play",
    title="The Rise of Passing Efficiency in the NFL",
    show_trend=True
)

fig.update_layout(
    xaxis_title="Season",
    yaxis_title="EPA per Pass Play",
)
fig.show()


## 📈 The Key Insight

**Look at that trend!**

Since 2010, passing efficiency has skyrocketed. The upward slope is undeniable.

This confirms our hypothesis:

> **To predict the Super Bowl, we MUST model the Quarterback.**

The QB is the most impactful position in modern football. The data proves it.


---

## Bonus: Pass Rate Over Time

Teams aren't just passing *better* — they're passing *more*:


In [ ]:
# Calculate pass rate by season
play_type_by_season = (
    pbp
    .filter(pl.col("play_type").is_in(["pass", "run"]))
    .group_by("season")
    .agg([
        pl.len().alias("total_plays"),
        (pl.col("play_type") == "pass").sum().alias("pass_plays")
    ])
    .with_columns(
        (pl.col("pass_plays") / pl.col("total_plays") * 100).alias("pass_rate")
    )
    .sort("season")
    .collect()
)

print("Pass Rate by Season:")
play_type_by_season.select(["season", "pass_rate"])


In [ ]:
# Visualize pass rate trend
fig = px.line(
    play_type_by_season.to_pandas(),
    x="season",
    y="pass_rate",
    title="NFL Pass Rate Over Time",
    markers=True
)
apply_dark_theme(fig)
fig.update_layout(
    xaxis_title="Season",
    yaxis_title="Pass Rate (%)"
)
fig.show()


## Next Steps

We've proven the NFL is a passing league. Now we need to:

1. **Get external validation** — Madden ratings as a proxy for talent
2. **Clean our data** — Player name matching across datasets
3. **Build our prediction model**

Continue to **[03_madden_oracle.ipynb](03_madden_oracle.ipynb)** →
